In [1]:
import os
import hashlib
import pandas as pd
import re
import json
from jsonpath_ng import jsonpath, parse
import datetime

## 1. json结构探查
对不同的json文件结构进行查看。由于json文件可能存在多层嵌套的结构，可能会增加读取的难度，需要用不同的方法尝试读取。
- 这里基本的印象是相同文件夹下的json文件内容的格式是一致的，因此在检查数据结构时我们只需要查看同一文件夹下的一个json文件的格式即可。
- 文件夹中可能会存在一些额外的json文件（例如misc.json、charalist.json等），用于数据库的属性设定。鉴于它们与同文件夹下的接送文件格式可能不太统一，我手动将其移动至others文件夹，之后处理。（ps.之后的数据探查表明这些文件多数为重复文本内容，故将所有文件移除）
- 鉴于json的多层嵌套结构，我有以下的想法：
> - 直接读取所有json文本转化为str，根据正则匹配关键字进行字段提取;
> - 基于pandas，直接尝试载入json。使用json_normalize，控制参数max_level实现逐层的解构;
> - 数据库方向的工具提取
- 最初的打算是使用json_normalize，也实现了基本的数据提取（傻瓜式暴力拆解）。但后来了解到普遍的做法是**使用jsonpath进行文本匹配**（类似于爬虫）即可。看来我还是图样，于是我使用jsonpath再进行了一次流程。
> - jsonpath是很强大的工具，具体参考：https://www.jianshu.com/p/3f5b9cc88bde
> - 这里使用json_normalize是为了调用pandas进行读取json的尝试，或许有更好的方法，仅以此抛砖引玉

当成功提取数据后，其格式为列表，那么便可依照以下逻辑进行存储：
- 按照路径得到json文件名，存入字段中；
- 根据路径，以13种不同的语言关键字（'zhs', 'zht', 'jp', 'ko', 'en', 'de', 'es', 'fr', 'pt', 'ru', 'id', 'th', 'vi'），分别将该json文件中的不同语言文本存入13个文件夹中，输出格式为txt；
- txt文件的处理方式和之前的Genshin_data.ipynb中的流程相同。当然也可以在上一步中选择不分别写入txt文件中直接用jsonpath返回的列表写入jsonl，此处仅是一种流程，即——尝试读取json文件数据，并使用pandas做数据分析。

### 1.1 json数据结构查看

In [2]:
# 部分结构简单的json文件可以直接通过pandas读取
pd.read_json(r'data\others\misc.json')

,talents,voices,story,constellations
zhs,天赋,语音,故事,命之座
zht,天賦,語音,故事,命之座
jp,天賦,ボイス,ストーリー,命ノ星座
ko,특성,음성,스토리,운명의 자리
en,Talents,Voice-Over,Story,Constellations
de,Talente,Stimme,Hintergrund,Sternbilder
es,Talentos,Voz,Historia,Constelaciones
fr,Aptitudes,Répliques,Histoire,Constellations
pt,Talentos,Voz,História,Constelação
ru,Таланты,Озвучивание,История,Созвездия


In [3]:
# 简单的行列转换即可获得按照列名排列的多语言文本数据
pd.read_json(r'data\others\misc.json').T.reset_index()

,index,zhs,zht,jp,ko,en,de,es,fr,pt,ru,id,th,vi
0,talents,天赋,天賦,天賦,특성,Talents,Talente,Talentos,Aptitudes,Talentos,Таланты,Talenta,พรสวรรค์,Thiên Phú
1,voices,语音,語音,ボイス,음성,Voice-Over,Stimme,Voz,Répliques,Voz,Озвучивание,Suara,เสียงพากย์,Lồng Tiếng
2,story,故事,故事,ストーリー,스토리,Story,Hintergrund,Historia,Histoire,História,История,Kisah,เนื้อเรื่อง,Câu Chuyện
3,constellations,命之座,命之座,命ノ星座,운명의 자리,Constellations,Sternbilder,Constelaciones,Constellations,Constelação,Созвездия,Konstelasi,กลุ่มดาว,Cung Mệnh


In [4]:
# 也有多层嵌套的结构,以下为例1
path_test0 = 'data/charadata/albedo/voice-lines.json'

df_test0 = pd.read_json(path_test0)
df_test0

,title,line
0,"{'zhs': '初次见面…', 'zht': '初次見面…', 'jp': '初めまして…...",{'zhs': '我是阿贝多，西风骑士团首席炼金术士。我对你身上星海的气息很感兴趣，想要就近...
1,"{'zhs': '闲聊·写生', 'zht': '閒聊·寫生', 'jp': '世間話·写生...","{'zhs': '风景不错，趁着休息，就把这一幕画下来吧。', 'zht': '風景不錯，趁..."
2,"{'zhs': '闲聊·探究', 'zht': '閒聊·深究', 'jp': '世間話·探究...","{'zhs': '「世界的真相」…究竟是什么呢？', 'zht': '「世界的真相」…究竟是..."
3,"{'zhs': '闲聊·怀旧', 'zht': '閒聊·懷舊', 'jp': '世間話·過去...","{'zhs': '用自己的双脚丈量土地，将「未知」变为知识。…这种感觉，真怀念啊。', 'z..."
4,"{'zhs': '下雨的时候…', 'zht': '下雨的時候…', 'jp': '雨の日…...","{'zhs': '土壤湿润，适合采集爆爆瓜的根。没事，我来动手就可以。', 'zht': '..."
...,...,...
60,"{'zhs': '普通受击·其一', 'zht': '普通受擊·其一', 'jp': 'ダメ...","{'zhs': '阈值以内…', 'zht': '閾值以內…', 'jp': 'しきい値内だ..."
61,"{'zhs': '重受击·其一', 'zht': '重受擊·其一', 'jp': '重ダメー...","{'zhs': '失算了…', 'zht': '失算了…', 'jp': 'ミスしたか…',..."
62,"{'zhs': '加入队伍·其一', 'zht': '加入隊伍·其一', 'jp': 'チー...","{'zhs': '了解。', 'zht': '瞭解。', 'jp': '了解。', 'ko'..."
63,"{'zhs': '加入队伍·其二', 'zht': '加入隊伍·其二', 'jp': 'チー...","{'zhs': '是拓展课题吗？', 'zht': '是拓展課題嗎？', 'jp': '課外..."


In [5]:
# 例2，这里的name列，包含字典
path_test1 = 'data/others/charalist.json'

df_test1 = pd.read_json(path_test1)
df_test1

,id,name
0,albedo,"{'zhs': '阿贝多', 'zht': '阿貝多', 'jp': 'アルベド', 'ko..."
1,aloy,"{'zhs': '埃洛伊', 'zht': '亞蘿伊', 'jp': 'アーロイ', 'ko..."
2,ambor,"{'zhs': '安柏', 'zht': '安柏', 'jp': 'アンバー', 'ko':..."
3,ayaka,"{'zhs': '神里绫华', 'zht': '神里綾華', 'jp': '神里綾華', '..."
4,ayato,"{'zhs': '神里绫人', 'zht': '神里綾人', 'jp': '神里綾人', '..."
5,barbara,"{'zhs': '芭芭拉', 'zht': '芭芭拉', 'jp': 'バーバラ', 'ko..."
6,beidou,"{'zhs': '北斗', 'zht': '北斗', 'jp': '北斗', 'ko': '..."
7,bennett,"{'zhs': '班尼特', 'zht': '班尼特', 'jp': 'ベネット', 'ko..."
8,candace,"{'zhs': '坎蒂丝', 'zht': '坎蒂絲', 'jp': 'キャンディス', '..."
9,chongyun,"{'zhs': '重云', 'zht': '重雲', 'jp': '重雲', 'ko': '..."


In [6]:
# 可以以使用json_normalize读取json达到同样效果
with open(path_test1, 'r', encoding='utf-8') as file:
    json_data_test = json.load(file)

df_test2 = pd.json_normalize(json_data_test, max_level=0)
df_test2

,id,name
0,albedo,"{'zhs': '阿贝多', 'zht': '阿貝多', 'jp': 'アルベド', 'ko..."
1,aloy,"{'zhs': '埃洛伊', 'zht': '亞蘿伊', 'jp': 'アーロイ', 'ko..."
2,ambor,"{'zhs': '安柏', 'zht': '安柏', 'jp': 'アンバー', 'ko':..."
3,ayaka,"{'zhs': '神里绫华', 'zht': '神里綾華', 'jp': '神里綾華', '..."
4,ayato,"{'zhs': '神里绫人', 'zht': '神里綾人', 'jp': '神里綾人', '..."
5,barbara,"{'zhs': '芭芭拉', 'zht': '芭芭拉', 'jp': 'バーバラ', 'ko..."
6,beidou,"{'zhs': '北斗', 'zht': '北斗', 'jp': '北斗', 'ko': '..."
7,bennett,"{'zhs': '班尼特', 'zht': '班尼特', 'jp': 'ベネット', 'ko..."
8,candace,"{'zhs': '坎蒂丝', 'zht': '坎蒂絲', 'jp': 'キャンディス', '..."
9,chongyun,"{'zhs': '重云', 'zht': '重雲', 'jp': '重雲', 'ko': '..."


In [7]:
# 针对name列的二次拆解
pd.json_normalize(df_test2.name, max_level=0)

,zhs,zht,jp,ko,en,de,es,fr,pt,ru,id,th,vi
0,阿贝多,阿貝多,アルベド,알베도,Albedo,Albedo,Albedo,Albedo,Albedo,Альбедо,Albedo,Albedo,Albedo
1,埃洛伊,亞蘿伊,アーロイ,에일로이,Aloy,Aloy,Aloy,Aloy,Aloy,Элой,Aloy,Aloy,Aloy
2,安柏,安柏,アンバー,엠버,Amber,Amber,Amber,Amber,Amber,Эмбер,Amber,Amber,Amber
3,神里绫华,神里綾華,神里綾華,카미사토 아야카,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Аяка,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka
4,神里绫人,神里綾人,神里綾人,카미사토 아야토,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Аято,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato
5,芭芭拉,芭芭拉,バーバラ,바바라,Barbara,Barbara,Bárbara,Barbara,Barbara,Барбара,Barbara,Barbara,Barbara
6,北斗,北斗,北斗,북두,Beidou,Beidou,Beidou,Beidou,Beidou,Бэй Доу,Beidou,Beidou,Beidou
7,班尼特,班尼特,ベネット,베넷,Bennett,Bennett,Bennett,Bennett,Bennett,Беннет,Bennett,Bennett,Bennett
8,坎蒂丝,坎蒂絲,キャンディス,캔디스,Candace,Candace,Candace,Candace,Candace,Кандакия,Candace,Candace,Candace
9,重云,重雲,重雲,중운,Chongyun,Chongyun,Chongyun,Chongyun,Chongyun,Чун Юнь,Chongyun,Chongyun,Chongyun


In [8]:
# 控制max_level参数进行深一层的解构
pd.json_normalize(json_data_test, max_level=1)

,id,name.zhs,name.zht,name.jp,name.ko,name.en,name.de,name.es,name.fr,name.pt,name.ru,name.id,name.th,name.vi
0,albedo,阿贝多,阿貝多,アルベド,알베도,Albedo,Albedo,Albedo,Albedo,Albedo,Альбедо,Albedo,Albedo,Albedo
1,aloy,埃洛伊,亞蘿伊,アーロイ,에일로이,Aloy,Aloy,Aloy,Aloy,Aloy,Элой,Aloy,Aloy,Aloy
2,ambor,安柏,安柏,アンバー,엠버,Amber,Amber,Amber,Amber,Amber,Эмбер,Amber,Amber,Amber
3,ayaka,神里绫华,神里綾華,神里綾華,카미사토 아야카,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Аяка,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka
4,ayato,神里绫人,神里綾人,神里綾人,카미사토 아야토,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Аято,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato
5,barbara,芭芭拉,芭芭拉,バーバラ,바바라,Barbara,Barbara,Bárbara,Barbara,Barbara,Барбара,Barbara,Barbara,Barbara
6,beidou,北斗,北斗,北斗,북두,Beidou,Beidou,Beidou,Beidou,Beidou,Бэй Доу,Beidou,Beidou,Beidou
7,bennett,班尼特,班尼特,ベネット,베넷,Bennett,Bennett,Bennett,Bennett,Bennett,Беннет,Bennett,Bennett,Bennett
8,candace,坎蒂丝,坎蒂絲,キャンディス,캔디스,Candace,Candace,Candace,Candace,Candace,Кандакия,Candace,Candace,Candace
9,chongyun,重云,重雲,重雲,중운,Chongyun,Chongyun,Chongyun,Chongyun,Chongyun,Чун Юнь,Chongyun,Chongyun,Chongyun


通过json_normalize，我使用比较简单（暴力）的拆解，根据逐个文件的结构规则设置方法进行读取，获得了最终的数据框并将其储存为jsonl。这是我最初的基本思路，之后执行的是基于jsonpath的做法。

## 2. 基于jsonpath进行json转txt
- 根据artedata,charadata,weaponsdata,others四个文件夹，依次遍历该文件夹目录下的json文件。
- 对每一个json文件进行13次遍历提取不同语言的文本，并依次存入对应13个文件夹中。

In [67]:
# rootpath_output = os.path.join(os.getcwd(),'data')
root_directory = os.path.join(os.getcwd(), "data")
# root_directory
language_columns = ['zhs', 'zht', 'jp', 'ko', 'en', 'de', 'es', 'fr', 'pt', 'ru', 'id', 'th', 'vi']

# 创建一个所有需要遍历的json文件路径列表，以作后用
all_files_path = []

def json_to_txt(path, file_name, target_language, from_file, id_check=False):
    # 编写 JSONPath 查询表达式来获取所有值
    jsonpath_expr = parse(f'$..{target_language}')

    with open(path,'r',encoding = 'utf-8') as file:
        json_data = json.load(file)

    # 使用查询表达式在 JSON 数据中查找匹配的内容
    matches = [match.value for match in jsonpath_expr.find(json_data)]
    # id_check参数在此处专门处理读取weaponsdata时出现的bug，详见下方
    if(id_check == True):
        matches = matches[1:]

    # 由于许多转义符数量的不对齐，导致转为txt文件后数据质量很差，在此处理转义符
    for i in range(len(matches)):
        matches[i] = re.sub('\n', ' ', matches[i])

    # 由于[artedata,charadata,weaponsdata,others]四个文件夹中的json文件可能出现重名的情况，以from_file加以区分
    file_name = re.sub('.json', f'_{from_file}_{target_language}.txt', file_name)

    path_output = os.path.join(root_directory,f'{target_language}\\{file_name}') 
    # 打开文件以写入模式
    with open(path_output, 'w', encoding='utf-8') as file:
        for value in matches:
            file.write(value + '\n')

# json_to_txt(json_data,'voice-lines.json','zhs')

### 2.1 artedata

In [68]:
path_artedata = os.path.join(root_directory, "artedata")
for file in os.listdir(path_artedata):
    if file.endswith('.json'):
        path = os.path.join(path_artedata, file)
        # print(f'path:{path}')
        all_files_path.append(path)
        print(f'正在处理文件:{file}')
        for column in language_columns:
            json_to_txt(path,file,column,"artedata")

正在处理文件:10001.json
正在处理文件:10002.json
正在处理文件:10003.json
正在处理文件:10004.json
正在处理文件:10005.json
正在处理文件:10006.json
正在处理文件:10007.json
正在处理文件:10008.json
正在处理文件:10009.json
正在处理文件:10010.json
正在处理文件:10011.json
正在处理文件:10012.json
正在处理文件:10013.json
正在处理文件:14001.json
正在处理文件:14002.json
正在处理文件:14003.json
正在处理文件:14004.json
正在处理文件:15001.json
正在处理文件:15002.json
正在处理文件:15003.json
正在处理文件:15004.json
正在处理文件:15005.json
正在处理文件:15006.json
正在处理文件:15007.json
正在处理文件:15008.json
正在处理文件:15009.json
正在处理文件:15010.json
正在处理文件:15011.json
正在处理文件:15012.json
正在处理文件:15013.json
正在处理文件:15014.json
正在处理文件:15015.json
正在处理文件:15016.json
正在处理文件:15017.json
正在处理文件:15018.json
正在处理文件:15019.json
正在处理文件:15020.json
正在处理文件:15021.json
正在处理文件:15022.json
正在处理文件:15023.json
正在处理文件:15024.json
正在处理文件:15025.json
正在处理文件:15026.json


### 2.2 charadata

In [69]:
path_charadata = os.path.join(root_directory, 'charadata')
for folder in os.listdir(path_charadata):
    path_folder = os.path.join(path_charadata, folder)
    # print(folder)
    for json_file in os.listdir(path_folder):
        if json_file.endswith('.json'):
            path = os.path.join(path_folder, json_file)
            all_files_path.append(path)
            print(f'正在处理文件:{json_file}')
            # print(json_file)
            for column in language_columns:
                json_to_txt(path,json_file,column,folder)

正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正在处理文件:constellations.json
正在处理文件:general.json
正在处理文件:stories.json
正在处理文件:talents.json
正在处理文件:voice-lines.json
正

### 2.3 weaponsdata
这里出现了bug，原因是关键词匹配问题。该文件中的json文件包含的'id'字段，既有表示'idenfication'的'id'字段，也有表示印地语的'id'字段，导致匹配时出现问题

In [70]:
path_weaponsdata = os.path.join(root_directory, 'weaponsdata')
for folder in os.listdir(path_weaponsdata):
    path_folder = os.path.join(path_weaponsdata, folder)
    print(f'当前文件夹：{folder}')
    for json_file in os.listdir(path_folder):
        if json_file.endswith('.json'):
            path = os.path.join(path_folder, json_file)
            all_files_path.append(path)
            print(f'正在处理文件:{json_file}')
            for column in language_columns:
                if(column == 'id'):
                    json_to_txt(path,json_file,column,folder,True)
                else:
                    json_to_txt(path,json_file,column,folder)

当前文件夹：bow
正在处理文件:amos.json
正在处理文件:arakalari.json
正在处理文件:arjuna.json
正在处理文件:ayus.json
正在处理文件:bakufu.json
正在处理文件:blackrock.json
正在处理文件:crowfeather.json
正在处理文件:curve.json
正在处理文件:dvalin.json
正在处理文件:exotic.json
正在处理文件:fallensun.json
正在处理文件:fin.json
正在处理文件:fleurfair.json
正在处理文件:fossil.json
正在处理文件:hardwood.json
正在处理文件:hunters-d-d-d.json
正在处理文件:hunters-d-d.json
正在处理文件:hunters-d.json
正在处理文件:hunters.json
正在处理文件:kirin.json
正在处理文件:maria.json
正在处理文件:msg.json
正在处理文件:nachtblind.json
正在处理文件:narukami.json
正在处理文件:old.json
正在处理文件:outlaw.json
正在处理文件:predator.json
正在处理文件:proto.json
正在处理文件:recluse.json
正在处理文件:sling.json
正在处理文件:theocrat.json
正在处理文件:troupe.json
正在处理文件:viridescent.json
正在处理文件:widsith.json
正在处理文件:worldbane.json
正在处理文件:zephyrus.json
当前文件夹：catalyst
正在处理文件:amber.json
正在处理文件:apprentice-d-d.json
正在处理文件:apprentice-d.json
正在处理文件:apprentice.json
正在处理文件:arakalari.json
正在处理文件:bakufu.json
正在处理文件:blackrock.json
正在处理文件:dvalin.json
正在处理文件:everfrost.json
正在处理文件:exotic.json
正在处理文件:fossil.json
正在处理文件:fourwinds.

## 3. txt数据载入

### 3.1 载入方法

In [71]:
root_directory = os.path.join(os.getcwd(), "data")

# 数据读取
def process_folder(folder):
    folder_path = os.path.join(root_directory, folder)
    all_files_data = []

    # 遍历文件夹中的所有txt文件
    for txt_file in os.listdir(folder_path):
        if txt_file.endswith(".txt"):
            file_path = os.path.join(folder_path, txt_file)
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
            
            for idx, line in enumerate(lines):
                cleaned_line = line.strip()
                # 设置zh_text_md5字段作为标识
                md5_hash = hashlib.md5(cleaned_line.encode()).hexdigest() 
                
                #正则化文件名，便于之后的merge操作
                file_name = re.sub(f'_{folder}.txt', '.txt', txt_file)
                if(folder != 'zhs'):
                    all_files_data.append({
                        '文件名': file_name,
                        '行号': idx + 1,
                        f'文本_{folder}': cleaned_line
                    })
                else:
                    all_files_data.append({
                        '文件名': file_name,
                        '行号': idx + 1,
                        'zh_text_md5': md5_hash,
                        f'文本_{folder}': cleaned_line
                    })
    return all_files_data
            
df_zhs = pd.DataFrame(process_folder('zhs')) #简中
df_zht = pd.DataFrame(process_folder('zht')) #繁中
df_de = pd.DataFrame(process_folder('de'))   #德语
df_en = pd.DataFrame(process_folder('en'))   #英
df_es = pd.DataFrame(process_folder('es'))   #西
df_fr = pd.DataFrame(process_folder('fr'))   #法
df_id = pd.DataFrame(process_folder('id'))   #印地
df_jp = pd.DataFrame(process_folder('jp'))   #日
df_ko = pd.DataFrame(process_folder('ko'))   #韩
df_pt = pd.DataFrame(process_folder('pt'))   #葡
df_ru = pd.DataFrame(process_folder('ru'))   #俄
df_th = pd.DataFrame(process_folder('th'))   #泰
df_vi = pd.DataFrame(process_folder('vi'))   #越南


### 3.2 载入后情况

In [72]:
# 可以看到底下的数据有一部分还是重复，需要做一些简单的去重
df_zhs

,文件名,行号,zh_text_md5,文本_zhs
0,10001_artedata.txt,1,dd608512c989e96decc20d4a4c53156b,行者之心
1,10001_artedata.txt,2,0ec39cfc1b06e6b5ac62dc7e689b1d31,攻击力提高18%。
2,10001_artedata.txt,3,86a111c4a6f097e7354b0616f085d287,重击的暴击率提升30%。
3,10001_artedata.txt,4,9da66b6278278619be64049bbd97d9a6,故人之心
4,10001_artedata.txt,5,a55867ecfda45425ca6ad34c1cf33b55,玉蓝色的小花，花茎上扎着谁人的丝带。
...,...,...,...,...
13060,zephyrus_sword.txt,9,e5d6d4d879ba5db47ae21f01962e4a96,攻击造成暴击时，有<color=#99FFFFFF>80%</color>的几率产生少量元素...
13061,zephyrus_sword.txt,10,44a59492cb7ee50266441565a92a82b1,顺风而行
13062,zephyrus_sword.txt,11,c842a5cf606543f2133250c0b9dd95da,攻击造成暴击时，有<color=#99FFFFFF>90%</color>的几率产生少量元素...
13063,zephyrus_sword.txt,12,44a59492cb7ee50266441565a92a82b1,顺风而行


In [73]:
df_zht

,文件名,行号,文本_zht
0,10001_artedata.txt,1,行者之心
1,10001_artedata.txt,2,攻擊力提高18%。
2,10001_artedata.txt,3,重擊的暴擊率提升30%。
3,10001_artedata.txt,4,故人之心
4,10001_artedata.txt,5,玉藍色的小花，花莖上繫著誰人的絲帶。
...,...,...,...
13060,zephyrus_sword.txt,9,攻擊造成暴擊時，有<color=#99FFFFFF>80%</color>的機率產生少量元素...
13061,zephyrus_sword.txt,10,順風而行
13062,zephyrus_sword.txt,11,攻擊造成暴擊時，有<color=#99FFFFFF>90%</color>的機率產生少量元素...
13063,zephyrus_sword.txt,12,順風而行


In [74]:

df_en

,文件名,行号,文本_en
0,10001_artedata.txt,1,Resolution of Sojourner
1,10001_artedata.txt,2,ATK +18%.
2,10001_artedata.txt,3,Increases Charged Attack CRIT Rate by 30%.
3,10001_artedata.txt,4,Heart of Comradeship
4,10001_artedata.txt,5,A small cerulean flower with somebody's ribbon...
...,...,...,...
13060,zephyrus_sword.txt,9,CRIT Hits have a <color=#99FFFFFF>80%</color> ...
13061,zephyrus_sword.txt,10,Windfall
13062,zephyrus_sword.txt,11,CRIT Hits have a <color=#99FFFFFF>90%</color> ...
13063,zephyrus_sword.txt,12,Windfall


### 3.3 数据框合并

In [82]:
# 数据合并
df_testmerge = df_zhs

df_testmerge = df_testmerge.merge(df_zht, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_de, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_en, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_es, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_fr, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_id, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_jp, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_ko, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_pt, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_ru, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_th, on=['文件名', '行号'], how='outer')
df_testmerge = df_testmerge.merge(df_vi, on=['文件名', '行号'], how='outer')

df_testmerge

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
0,10001_artedata.txt,1,dd608512c989e96decc20d4a4c53156b,行者之心,行者之心,Fernweh,Resolution of Sojourner,Corazón del Viajero,Cœur du Voyageur,Resolution of Sojourner,旅人の心,행자의 마음,Resolução de Sojourner,Решимость временщика,Resolution of Sojourner,Trái Tim Hành Giả
1,10001_artedata.txt,2,0ec39cfc1b06e6b5ac62dc7e689b1d31,攻击力提高18%。,攻擊力提高18%。,Erhöht den ANG um 18 %.,ATK +18%.,ATQ +18%.,Augmente l'ATQ de 18 %.,ATK +18%.,攻撃力+18%。,공격력+18%,ATQ +18%.,Увеличивает силу атаки на 18%.,เพิ่มพลังโจมตี 18%,Tấn công tăng 18%
2,10001_artedata.txt,3,86a111c4a6f097e7354b0616f085d287,重击的暴击率提升30%。,重擊的暴擊率提升30%。,Erhöht die KT heftiger Schläge um 30 %.,Increases Charged Attack CRIT Rate by 30%.,La Prob. CRIT de los Ataques Cargados aumenta ...,Augmente le taux CRIT des attaques chargées de...,Meningkatkan 30% CRIT Rate dari Charged Attack.,重撃の会心率+30%。,강공격의 치명타 확률+30%,Aumenta a Taxa Crítica do Ataque Carregado em ...,Увеличивает шанс крит. попадания заряженной ат...,เพิ่มอัตราคริของ การชาร์จโจมตีขึ้น 30%,Tỷ lệ bạo kích của trọng kích tăng 30%
3,10001_artedata.txt,4,9da66b6278278619be64049bbd97d9a6,故人之心,故人之心,Sinn der Kameradschaft,Heart of Comradeship,Corazón de la Amistad,Fleur du Voyageur,Heart of Comradeship,故人の心,옛 벗의 마음,Coração de Camaradagem,Сердце товарищества,Heart of Comradeship,Trái Tim Của Cố Nhân
4,10001_artedata.txt,5,a55867ecfda45425ca6ad34c1cf33b55,玉蓝色的小花，花茎上扎着谁人的丝带。,玉藍色的小花，花莖上繫著誰人的絲帶。,Ein hellblaues Blümchen mit einem kleinen Band...,A small cerulean flower with somebody's ribbon...,Una flor azul celeste en cuyo tallo anudaron u...,Une petite fleur bleue avec un ruban attaché à...,"Bunga kecil berwarna biru mutiara, dengan sehe...",藍色の小花、花茎に誰かのリボンがついている。,푸른빛의 작은 꽃. 꽃줄기에 오래된 누군가의 리본이 묶여있다,"Uma pequena flor azul jade, cuja fita está ama...",Маленький цветок с сапфировыми лепестками и пр...,ดอกไม้ขนาดเล็กที่มีสีของหยกสีฟ้า ที่ก้านมีริบบ...,"Bông hoa nhỏ màu xanh ngọc, trên thân có buộc ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13060,zephyrus_sword.txt,9,e5d6d4d879ba5db47ae21f01962e4a96,攻击造成暴击时，有<color=#99FFFFFF>80%</color>的几率产生少量元素...,攻擊造成暴擊時，有<color=#99FFFFFF>80%</color>的機率產生少量元素...,Erschafft bei einem KT mit einer Wahrscheinlic...,CRIT Hits have a <color=#99FFFFFF>80%</color> ...,Los Golpes CRIT tienen un <color=#99FFFFFF>80%...,Les coups CRIT des attaques ont <color=#99FFFF...,Serangan CRIT memiliki <color=#99FFFFFF>80%</c...,会心攻撃をした時、<color=#99FFFFFF>80%</color>の確率で少量の元素...,치명타 시 <color=#99FFFFFF>80%</color>의 확률로 소량의 원소...,Os Acertos Críticos possuem <color=#99FFFFFF>8...,Критические атаки имеют <color=#99FFFFFF>80%</...,เมื่อโจมตีคริติคอล จะมีโอกาส <color=#99FFFFFF>...,Khi tạo thành bạo kích có tỷ lệ <color=#99FFFF...
13061,zephyrus_sword.txt,10,44a59492cb7ee50266441565a92a82b1,顺风而行,順風而行,Folge dem Wind,Windfall,Avanza con el viento,Charge critique,Wind Companion,追い風が吹く,바람과 함께,Companhia do Vento,Дружественный бриз,Windfall,Đi Cùng Với Gió
13062,zephyrus_sword.txt,11,c842a5cf606543f2133250c0b9dd95da,攻击造成暴击时，有<color=#99FFFFFF>90%</color>的几率产生少量元素...,攻擊造成暴擊時，有<color=#99FFFFFF>90%</color>的機率產生少量元素...,Erschafft bei einem KT mit einer Wahrscheinlic...,CRIT Hits have a <color=#99FFFFFF>90%</color> ...,Los Golpes CRIT tienen un <color=#99FFFFFF>90%...,Les coups CRIT des attaques ont <color=#99FFFF...,Serangan CRIT memiliki <color=#99FFFFFF>90%</c...,会心攻撃をした時、<color=#99FFFFFF>90%</color>の確率で少量の元素...,치명타 시 <color=#99FFFFFF>90%</color>의 확률로 소량의 원소...,Os Acertos Críticos possuem <color=#99FFFFFF>9...,Критические атаки имеют <color=#99FFFFFF>90%</...,เมื่อโจมตีคริติคอล จะมีโอกาส <color=#99FFFFFF>...,Khi tạo thành bạo kích có tỷ lệ <color=#99FFFF...
13063,zephyrus_sword.txt,12,44a59492cb7ee50266441565a92a82b1,顺风而行,順風而行,Folge dem Wind,Windfall,Avanza con el viento,Charge critique,Wind Companion,追い風が吹く,바람과 함께,Companhia do Vento,Дружественный бриз,Windfall,Đi Cùng Với Gió


In [76]:

df_testmerge[df_testmerge['文件名'] == 'stories_chongyun.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
3296,stories_chongyun.txt,1,59d868f1db17425900fe146eee7cc65a,角色详细,角色詳細,Figurendetails,Character Details,Detalles del personaje,Info. personnage,Rincian Karakter,キャラクター詳細,캐릭터 상세정보,Detalhes,О персонаже,ข้อมูลตัวละคร,Miêu Tả Nhân Vật
3297,stories_chongyun.txt,2,284d0651c6e74bc8e9c5b1c16f977407,璃月港悠长的历史中，各种妖邪异闻从未断绝。无论真相如何，总要有人出面解决。 重云出生于知名...,璃月港悠長的歷史中，各種妖邪異聞從未斷絕。無論真相如何，總要有人出面解決。 重雲出生於知名...,Liyues langer Geschichte hat es nie an Gerücht...,"Throughout its long history, Liyue has never h...",A lo largo de la rica historia de Liyue nunca ...,De tout temps ont circulé à Liyue nombre de ru...,"Sepanjang sejarahnya yang panjang, Liyue tidak...",歴史の長い璃月港では、魑魅魍魎に関する異聞も少なくなかった。真相はどうあれ、解決をする誰かが...,리월항의 오랜 역사에서 각종 요마에 관한 이상한 소문은 끊이질 않았다. 진실이 어찌...,"Ao longo da sua extensa história, Liyue nunca ...",В Ли Юэ всегда было полно мифов и легенд о все...,ประวัติศาสตร์อันแสนจะยาวนาน ของท่าเรือ Liyue น...,Liyue là vùng đất có bề dày lịch sử văn hóa lâ...
3298,stories_chongyun.txt,3,fe3f81a356073aa0ef5b0d29a7330ae1,角色故事1,角色故事1,Figurenhintergrund 1,Character Story 1,Historia del personaje (1),Histoire du personnage 1,Kisah Karakter 1,キャラクターストーリー1,캐릭터 스토리1,História 1,История персонажа 1,เรื่องราวของตัวละคร 1,Câu Chuyện Nhân Vật 1
3299,stories_chongyun.txt,4,28c45a86faa047804ad7c9b8a644cf63,重云初出茅庐时，璃月港内恰好流传着一桩怪谈。 苦主是一位豪门阔太，身份高到与七星都能说上一...,重雲初出茅廬時，璃月港內恰好流傳著一樁怪談。 苦主是一位豪門夫人，身分高到與七星都能說上一...,Als Chongyun sich in seiner Rolle als Exorzist...,When Chongyun first ventured out as an exorcis...,Cuando Chongyun hizo su primer encargo como ex...,Au début de la carrière de Chongyun courait à ...,Ketika Chongyun pertama kali bertualang sebaga...,重雲が駆け出しだった頃、ちょうど璃月港である怪談が流行っていた。 被害者は七星と直接話せる...,"중운이 퇴마사로 처음 세상에 나왔을 때, 리월항엔 한 가지 괴담이 돌고 있었다. ...",Quando Chongyun se aventurou pela primeira vez...,"По Ли Юэ ходили слухи о злом духе, когда Чун Ю...",ในขณะที่ Chongyun ยังเป็นเพียงเด็กไร้ประสบการณ...,Khi Chongyun vừa ra đời thì cảng Liyue đúng lú...
3300,stories_chongyun.txt,5,36123cf203f2e21976ce28c9c8aef0b5,角色故事2,角色故事2,Figurenhintergrund 2,Character Story 2,Historia del personaje (2),Histoire du personnage 2,Kisah Karakter 2,キャラクターストーリー2,캐릭터 스토리2,História 2,История персонажа 2,เรื่องราวของตัวละคร 2,Câu Chuyện Nhân Vật 2
3301,stories_chongyun.txt,6,8f9d93656f5261c3f20338a56c6919bb,事实上，重云的行事风格与「冰霜」之类，全然无关。 生来就是「纯阳之体」的他，总是为自己易热...,事實上，重雲的行事風格與「冰霜」之類，全然無關。 生來就是「純陽之體」的他，總是為自己易熱...,Allerdings könnte diese Bezeichnung nicht weit...,"However, calling his mannerisms and style ""fro...","Sin embargo, esa afirmación no podría estar má...",Cette réputation de personnage froid et détach...,"Sebenarnya, tidak ada kaitannya sama sekali an...",実のところ、重雲の行動スタイルは「氷霜」と何の関係もない。 生まれた時から「純陽の体」を持...,"사실, 중운의 행동 스타일은 「얼음과 서리」 따위와는 전혀 관계가 없다. 태어날 ...","Entretanto, chamar seus maneirismos e estilo d...",Такое сравнение было бы не вполне уместным. О...,ความจริงแล้วบุคลิกและสไตล์ของ Chongyun นั้นไม่...,"Trên thực tế, phong cách làm việc của Chongyun..."
3302,stories_chongyun.txt,7,2d6bbfbdc2962057bc47f74e3cad3c40,角色故事3,角色故事3,Figurenhintergrund 3,Character Story 3,Historia del personaje (3),Histoire du personnage 3,Kisah Karakter 3,キャラクターストーリー3,캐릭터 스토리3,História 3,История персонажа 3,เรื่องราวของตัวละคร 3,Câu Chuyện Nhân Vật 3
3303,stories_chongyun.txt,8,db61f036eac618eb00168d37e96ed206,「纯阳之体」不仅是重云方士之路上的一大障碍，也是日常生活中的大麻烦。 他对「阳」的存在非常...,「純陽之體」不僅是重雲方士之路上的一大障礙，也是日常生活中的大麻煩。 他對「陽」的存在非常...,Seine Yang-Lastigkeit ist nicht nur ein großes...,Chongyun's pure-yang spirit is not only a huge...,La positividad congénita de Chongyun no es sol...,Cette puissante énergie qui habite Chongyun n'...,"""Luapan energi Yang"" milik Chongyun bukan hany...",重雲にとって、「純陽の体」は方士への道の大きな障害である他、日常生活でも注意しなければならな...,중운에게 있어 「양(陽)의 체질」은 퇴마 시의 걸림돌이었을 뿐만 아니라 일상생활에서...,O Psíquico Yang Puro de Chongyun não é um gran...,Чистая энергия Ян мешает Чун Юню не только в р...,"""ร่างกายแห่งหยางบริสุทธิ์"" นั้นไม่เพียงแต่จะเป...",Thuần dương chi thể khô

## 4. 数据探查
可以看到数据存在一定程度的缺失，可能是部分语言的列在原json文件中不存在导致在json转换txt
> 经过排查，多数是json文件中的转义符数量可能存在不等导致

解决办法：
> 在json转txt之前用正则将所有转义符（如\n）替换，确保文本对齐

### 4.1 总体情况

In [21]:
df_testmerge.info

<bound method DataFrame.info of                         文件名   行号                       zh_text_md5  \
0        10001_artedata.txt    1  dd608512c989e96decc20d4a4c53156b   
1        10001_artedata.txt    2  0ec39cfc1b06e6b5ac62dc7e689b1d31   
2        10001_artedata.txt    3  86a111c4a6f097e7354b0616f085d287   
3        10001_artedata.txt    4  9da66b6278278619be64049bbd97d9a6   
4        10001_artedata.txt    5  a55867ecfda45425ca6ad34c1cf33b55   
...                     ...  ...                               ...   
13486  charalist_others.txt  108                               NaN   
13487  charalist_others.txt  109                               NaN   
13488  charalist_others.txt  110                               NaN   
13489  charalist_others.txt  111                               NaN   
13490  charalist_others.txt  112                               NaN   

                   文本_zhs              文本_zht  \
0                    行者之心                行者之心   
1               攻击力提高18%。    

In [22]:
# 缺失值分析
print("缺失值数量:\n", df_testmerge.isnull().sum())


缺失值数量:
 文件名             0
行号              0
zh_text_md5    86
文本_zhs         86
文本_zht         58
文本_de          58
文本_en          58
文本_es          58
文本_fr          56
文本_id           2
文本_jp          56
文本_ko          58
文本_pt          58
文本_ru          56
文本_th          58
文本_vi          58
dtype: int64


In [23]:
# 唯一值分析，其与数据框总行数的差值代表了简中文本的缺失量
print("zh_text_md5唯一值数量:", df_testmerge['zh_text_md5'].nunique())


zh_text_md5唯一值数量: 8936


In [37]:
# 检查具体某列的值分布
column_name = '文本_ru'
print(f"{column_name}的唯一值数量:", df_testmerge[column_name].nunique())

文本_ru的唯一值数量: 8882


In [25]:

df_null_zh_text_md5 = df_testmerge[df_testmerge['zh_text_md5'].isnull()]
df_null_zh_text_md5


,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
13405,aniki-d-d_claymore.txt,2,NaN,NaN,是最初的仙人與最後的仙獸的故事的另一個結局，是雙方和解的橋樑與脊柱。無鋒而有力。,Ein anderes Ende für die Geschichte des ersten...,The alternative ending to the story between th...,Era el final alternativo de la historia entre ...,Une épée sans lame mais puissante qui décrit l...,Akhir kisah yang berbeda dari cerita antara ad...,最初の仙人と最後の仙獣の物語のもう一つのエンディング、双方を和解に運ぶ懸け橋と柱である。峰は...,최초의 선인과 최후의 신수의 이야기의 또 다른 결말로 양자간 화해의 징검다리이다. ...,O final alternativo da história entre o primei...,Альтернативный конец истории первого Адепта и ...,เป็นเรื่องราวตอนจบที่สามารถเลือกได้ระหว่างจุดเ...,Là một kết cuộc khác của tiên nhân đầu tiên vớ...
13406,aniki-d-d_claymore.txt,3,NaN,NaN,測試,Test,Test,Prueba,Test,Ujian,テスト,테스트,Teste,Тест,ทดสอบ,Thử nghiệm
13407,aniki-d_claymore.txt,2,NaN,NaN,曾屬於魔神的玉璋。一度被擲入漆黑的海洋，以鎮服滔天的滄波。,"Ein Stück Jade, das einem Archon gehörte. Als ...",A jade greatsword that once belonged to an Arc...,Una horquilla que se transformó en una ballena...,Une grande épée en jade ayant utilisée par l'A...,Sebuah batu giok milik Archon yang dilemparkan...,かつて魔神に所有された美玉、一度荒波を静まらせるために漆黒の海に投げられた。,마신이 들고 있던 옥홀. 칠흑 같던 바다에 던져져 하늘을 뒤덮을 듯한 파도를 굴복시...,Um espadão de jade que pertenceu a um Arconte ...,"Нефритовый меч, который когда-то принадлежал а...",แผ่นหยกที่ครั้งหนึ่งเคยเป็นของเทพ ถูกโยนลงไปใน...,Từng là miếng ngọc bội thuộc về ma thần. Từng ...
13408,aniki-d_claymore.txt,3,NaN,NaN,,,,,,,,,,,,
13409,apprentice-d-d_catalyst.txt,2,NaN,NaN,紀念於山之際海之濱尋得安身之地的古代寶玉。如今仍隱隱散發古老的力量，守護國土安寧。,"Ein uralter Edelstein, der für die lange Suche...",An ancient gem that symbolizes the will of an ...,Una piedra preciosa que fue testigo del nacimi...,Une ancienne gemme commémorative qui symbolise...,Sebuah mutiara kuno yang melambangkan kehendak...,山の際と海の浜で居場所を見つけたことを記念するために作られた古代の宝玉。今でも古の力を放ち、...,산의 변두리와 바다의 가장 자리에서 삶의 터전을 마련한 걸 기리기 위해 만들어진 보...,Uma joia antiga que simboliza a vontade de um ...,"Древний драгоценный камень, символизирующий во...",อัญมณีโบราณที่แสดงถึงเจตจำนงของคนโบราณที่แสวงห...,Bảo ngọc cổ đại để kỷ niệm ranh giới giữa núi ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13486,charalist_others.txt,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yoimiya,NaN,NaN,NaN,NaN,NaN,NaN
13487,charalist_others.txt,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yunjin,NaN,NaN,NaN,NaN,NaN,NaN
13488,charalist_others.txt,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yun Jin,NaN,NaN,NaN,NaN,NaN,NaN
13489,charalist_others.txt,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zhongli,NaN,NaN,NaN,NaN,NaN,NaN


### 4.2 缺失排查
存在以下情况：

- json文件字段缺失（例如只有4种语言的字段），仅一例
- 缺失某种语言的字段（都是只缺了简体中文的文本内容），共十个文件
- 由于以上的原因而关联的文件，共两个文件

举例：
存在缺失文件共13个，经直接查看json原文件，情况应为对应语言关键字段缺失导致语言文本不对齐的情况。例如以下在json文件（路径'..\charadata\shougun\talents.json'）的字段：

```json
{
    "title": {
      "zhs": "暂缺",
      "jp": "暫くなし",
      "fr": "暂缺",
      "ru": "暂缺"
    },
    "description": {
      "zhs": "无法参与料理",
      "jp": "料理に参加できません",
      "fr": "无法参与料理",
      "ru": "无法参与料理"
    }
},
```
可以看到上述片段缺失了另外9种语言的字段，导致之后的文本不对齐。

鉴于文件数量不大，进行以下操作：
> 删除部分字段：对talents_shougun的json原文件进行操作
>
> 删除json字段缺失过多的文件:list_weaponsdata_others.txt, charalist_others.txt
>
> 根据繁体中文内容，手动填补简中内容

In [26]:
# 筛选出含有至少一个缺失值的所有行
df_with_missing_values = df_testmerge[df_testmerge.isnull().any(axis=1)]

# 从这些行中提取文件名列，并获取唯一值
unique_files_with_missing_values = df_with_missing_values['文件名'].unique()

# 打印结果
print("包含缺失值的文件名有：")
for file_name in unique_files_with_missing_values:
    print(file_name)


包含缺失值的文件名有：
talents_shougun.txt
aniki-d-d_claymore.txt
aniki-d_claymore.txt
apprentice-d-d_catalyst.txt
apprentice-d_catalyst.txt
blunt-d-d_sword.txt
blunt-d_sword.txt
gewalt-d_pole.txt
hunters-d-d-d_bow.txt
hunters-d-d_bow.txt
hunters-d_bow.txt
list_weaponsdata_others.txt
charalist_others.txt


In [38]:
# chardata文件夹下的shougun文件夹里的talents的情况
df_testmerge[df_testmerge['文件名'] == 'talents_shougun.txt']
# json原文件存在语言关键字段缺失

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
4879,talents_shougun.txt,1,1a8183bde59ff7069b456b444ecbb114,普通攻击·源流,普通攻擊·源流,Standardangriff: Ursprung,Normal Attack: Origin,Ataque Normal: Estilo Genryuu,Attaque normale : Prémisse,Normal Attack: Origin,通常攻撃·源流,일반 공격·원류,Ataque Normal: Origem,Исток,โจมตีปกติ: Origin,Tấn Công Thường - Origin
4880,talents_shougun.txt,2,457ee5055ee8fe20527e8f91d5f594c4,<color=#FFD780FF>普通攻击</color> 进行至多五段的连续枪击。 <c...,<color=#FFD780FF>普通攻擊</color> 進行至多五段的連續槍擊。 <c...,<color=#FFD780FF>Standardangriff</color> Führt...,<color=#FFD780FF>Normal Attack</color> Perform...,<color=#FFD780FF>Ataque Normal</color> Realiza...,<color=#FFD780FF>Attaque normale</color> Encha...,<color=#FFD780FF>Normal Attack</color> Melanca...,<color=#FFD780FF>通常攻撃</color> 槍による最大5段の連続攻撃を行う...,<color=#FFD780FF>일반 공격</color> 창으로 최대 5번 공격한다....,<color=#FFD780FF>Ataque Normal</color> Realiza...,<color=#FFD780FF>Обычная атака:</color> До пят...,<color=#FFD780FF>โจมตีปกติ</color> โจมตีด้วยหอ...,<color=#FFD780FF>Tấn Công Thường</color> Thực ...
4881,talents_shougun.txt,3,eaf82927d87662084b9aef9be4ae7d88,神变·恶曜开眼,神變·惡曜開眼,Wunder – Unheilvoller Blick,Transcendence: Baleful Omen,Trascendencia: Presagio maligno,Transcendance : Présage funeste,Transcendence: Baleful Omen,神変·悪曜開眼,초월·악요개안,Milagre: Luz Maligna,Превосходство: Зловещее знамение,Transcendence: Baleful Omen,Thần Biến - Ánh Chớp Tà Ác
4882,talents_shougun.txt,4,6a80efa4905a4e517d56982466c01b48,雷电将军展开净土的一角，对周围的敌人造成<color=#FFACFFFF>雷元素伤害</co...,雷電將軍展開淨土的一角，對周圍的敵人造成<color=#FFACFFFF>雷元素傷害</co...,Der Shougun Raiden offenbart einen kleinen Tei...,The Raiden Shogun unveils a shard of her Euthy...,La Shogun Raiden libera un fragmento de su eut...,La Shogun Raiden dévoile un fragment de son eu...,"Raiden Shogun membuka sebagian Euthymia-nya, m...",雷電将軍が浄土の一角を展開し、周囲の敵に<color=#FFACFFFF>雷元素ダメージ</...,라이덴 쇼군이 정토의 일각을 펼쳐 주변의 적에게 <color=#FFACFFFF>번개...,A Shogun Raiden revela um fragmento de sua Eut...,Сёгун Райдэн приоткрывает своё Царство и нанос...,โชกุน Raiden เปิดเผยเศษเสี้ยวของดินแดนอันบริสุ...,"Raiden Shogun khai mở Tịnh Thổ, gây <color=#FF..."
4883,talents_shougun.txt,5,d2b140341cfd640ae32bd656d02b84b2,奥义·梦想真说,奧義·夢想真說,Geheime Kunst – Musou Shinsetsu,Secret Art: Musou Shinsetsu,Técnica secreta: Verdad onírica,Art secret : Dogme d'idéal,Secret Art: Musou Shinsetsu,奥義·夢想真説,오의·몽상진설,Arte Secreta: Musou Shinsetsu,Тайное искусство: Мусо синсэцу,Secret Art: Musou Shinsetsu,Bí Kỹ - Mộng Tưởng Chân Thuyết
4884,talents_shougun.txt,6,97441c64f05e6cc428a2a60a1f464eac,汇聚万千真言，竭尽诸愿百眼之愿力，斩出粉碎一切诅咒的梦想一刀，造成<color=#FFACF...,匯聚萬千真言，竭盡諸願百眼之願力，斬出粉碎一切詛咒的夢想一刀，造成<color=#FFACF...,Der Shougun Raiden sammelt die Stärke der abso...,Gathering truths unnumbered and wishes uncount...,Reúne las infinitas verdades y consume su ambi...,Rassemblant des vérités innombrables et des so...,Dengan mengumpulkan kebenaran dan harapan yang...,無数の真言を集結し、諸願百目の願力を尽くす。全ての呪詛を切り裂く夢想の一太刀を繰り出し、<c...,수만 개의 진언을 모아 염원이 깃든 백안의 원력을 사용해 모든 저주를 베어버리는 몽...,Reunindo verdades não numeradas e desejos inco...,"Собрав несметные истины и бесчисленные мечты, ...",รวบรวมความจริงอันมากมายและความปรารถนานับไม่ถ้ว...,"Tích tụ ngàn vạn chân ngôn, dùng hết Ý Lực Ước..."
4885,talents_shougun.txt,7,a803d1a4127048336d8c6dd03de6bcbb,万千的愿望,萬千的願望,Unzählige Wünsche,Wishes Unnumbered,Aspiración infinita,Souhaits innombrables,Wishes Unnumbered,千万の願望,수천수만의 염원,Desejos Incontáveis,Бесчисленные мечты,Wishes Unnumbered,Ngàn Vạn Ước Nguyện
4886,talents_shougun.txt,8,7a724130e7b08954d88f6bc3d5ff1dd7,队伍中附近的角色获得元素晶球或元素微粒时，会为<color=#FFD780FF>诸愿百眼之轮...,隊伍中附近的角色獲得元素晶球或元素微粒時，會為<color=#FFD780FF>諸願百眼之輪...,Fügt <color=#FFD780FF>Charkra-Desiderate</colo...,When nearby party members gain Elemental Orbs ...,Los orbes o partículas elementales obtenidos p...,Lorsque les personnages de l'équipe à proximit...,Saat anggota party di sekitar memperoleh Eleme...,周囲にいるチーム中キャラクターが元素オーブまたは元素粒子を獲得した時、<color=#FFD...,근처에 있는 파티 내 캐릭터가 원소 구슬 또는 원소 입자를 획득하면 <color=

In [40]:
# 该文件为缺失简中字段
df_testmerge[df_testmerge['文件名'] == 'aniki-d-d_claymore.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
748,aniki-d-d_claymore.txt,1,db06c78d1e24cf708a14ce81c9b617ec,测试,異史,Anderer Verlauf,The Other Side,Historia Diferente,Histoire secondaire,Historia Diferente,異史,괴사,O Outro Lado,Другая Сторона,The Other Side,Dị Sử
13405,aniki-d-d_claymore.txt,2,NaN,NaN,是最初的仙人與最後的仙獸的故事的另一個結局，是雙方和解的橋樑與脊柱。無鋒而有力。,Ein anderes Ende für die Geschichte des ersten...,The alternative ending to the story between th...,Era el final alternativo de la historia entre ...,Une épée sans lame mais puissante qui décrit l...,Akhir kisah yang berbeda dari cerita antara ad...,最初の仙人と最後の仙獣の物語のもう一つのエンディング、双方を和解に運ぶ懸け橋と柱である。峰は...,최초의 선인과 최후의 신수의 이야기의 또 다른 결말로 양자간 화해의 징검다리이다. ...,O final alternativo da história entre o primei...,Альтернативный конец истории первого Адепта и ...,เป็นเรื่องราวตอนจบที่สามารถเลือกได้ระหว่างจุดเ...,Là một kết cuộc khác của tiên nhân đầu tiên vớ...
13406,aniki-d-d_claymore.txt,3,NaN,NaN,測試,Test,Test,Prueba,Test,Ujian,テスト,테스트,Teste,Тест,ทดสอบ,Thử nghiệm


In [41]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'aniki-d_claymore.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
749,aniki-d_claymore.txt,1,d41d8cd98f00b204e9800998ecf8427e,,砥厄魚,Urzeitliches Jadeschwert,Primordial Jade Greatsword,Espada de Jade Inicial,Grande épée de jade primordial,Primordial Jade Greatsword,美玉魚,고래 비녀,Espada de Jade Primordial,Древний Нефритовый Двуручный Меч,Primordial Jade Greatsword,Đề Ách Ngư
13407,aniki-d_claymore.txt,2,NaN,NaN,曾屬於魔神的玉璋。一度被擲入漆黑的海洋，以鎮服滔天的滄波。,"Ein Stück Jade, das einem Archon gehörte. Als ...",A jade greatsword that once belonged to an Arc...,Una horquilla que se transformó en una ballena...,Une grande épée en jade ayant utilisée par l'A...,Sebuah batu giok milik Archon yang dilemparkan...,かつて魔神に所有された美玉、一度荒波を静まらせるために漆黒の海に投げられた。,마신이 들고 있던 옥홀. 칠흑 같던 바다에 던져져 하늘을 뒤덮을 듯한 파도를 굴복시...,Um espadão de jade que pertenceu a um Arconte ...,"Нефритовый меч, который когда-то принадлежал а...",แผ่นหยกที่ครั้งหนึ่งเคยเป็นของเทพ ถูกโยนลงไปใน...,Từng là miếng ngọc bội thuộc về ma thần. Từng ...
13408,aniki-d_claymore.txt,3,NaN,NaN,,,,,,,,,,,,


In [42]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'apprentice-d-d_catalyst.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
753,apprentice-d-d_catalyst.txt,1,d41d8cd98f00b204e9800998ecf8427e,,定土玉圭,Urzeitliche Jadeinsignien,Primordial Jade Regalia,Regalia de Jade Primordial,Tablette de jade primordial,Primordial Jade Regalia,土定の玉,정토옥규,Presente de Jade Primordial,Древняя Нефритовая Регалия,Primordial Jade Regalia,Ngọc Định Thổ
13409,apprentice-d-d_catalyst.txt,2,NaN,NaN,紀念於山之際海之濱尋得安身之地的古代寶玉。如今仍隱隱散發古老的力量，守護國土安寧。,"Ein uralter Edelstein, der für die lange Suche...",An ancient gem that symbolizes the will of an ...,Una piedra preciosa que fue testigo del nacimi...,Une ancienne gemme commémorative qui symbolise...,Sebuah mutiara kuno yang melambangkan kehendak...,山の際と海の浜で居場所を見つけたことを記念するために作られた古代の宝玉。今でも古の力を放ち、...,산의 변두리와 바다의 가장 자리에서 삶의 터전을 마련한 걸 기리기 위해 만들어진 보...,Uma joia antiga que simboliza a vontade de um ...,"Древний драгоценный камень, символизирующий во...",อัญมณีโบราณที่แสดงถึงเจตจำนงของคนโบราณที่แสวงห...,Bảo ngọc cổ đại để kỷ niệm ranh giới giữa núi ...
13410,apprentice-d-d_catalyst.txt,3,NaN,NaN,,,,,,,,,,,,


In [43]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'apprentice-d_catalyst.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
754,apprentice-d_catalyst.txt,1,d41d8cd98f00b204e9800998ecf8427e,,散佚的敘事詩,Ballade des Verlusts,Lost Ballade,Balada Perdida,Ballade perdue,Lost Ballade,散逸した叙事詩,산실된 서사시,Poema Perdido,Утерянная Баллада,Lost Ballade,Bài Thơ Kể Chuyện
13411,apprentice-d_catalyst.txt,2,NaN,NaN,傳說中，追隨風之神的詩人踏遍了一千世界。歌集中記載了諸多奇景，飽含偉力。這些斷章有些丟失了，...,"Der Legende zufolge besuchte der Dichter, der ...",Legend has it that the bard who walked with th...,La leyenda dice que el bardo que caminó con el...,La légende dit qu'un barde avait parcouru des ...,"Menurut legenda, penyair yang berjalan bersama...",伝説では、風の神を追随する詩人が千の世界を踏破したらしい。その歌集には様々な奇景が記載されて...,전설에 의하면 바람의 신을 추종하던 시인이 전 세계를 모두 둘러보며 만들었다던 시가...,Diz a lenda que o bardo que andou com o Deus d...,"Легенды гласят, что бард, который ходил с архо...",ในตำนานเล่าว่านักกวีที่ติดตามเทพแห่งลม เดินทาง...,"Trong truyền thuyết, nhà thơ đi theo Phong Thầ..."
13412,apprentice-d_catalyst.txt,3,NaN,NaN,,,,,,,,,,,,


In [44]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'blunt-d-d_sword.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
992,blunt-d-d_sword.txt,1,db06c78d1e24cf708a14ce81c9b617ec,测试,憑虛,Gefäß der Leere,One Side,Destino Vacío,Destin du néant,One Side,虚構,공허한 운명,Um Lado,Одна Сторона,One Side,Hư Vô
13413,blunt-d-d_sword.txt,2,NaN,NaN,最初用兵器彌補失去的力量的仙人所使用的劍。斬殺過無數的妖魔，最終也終結了過去的同族。,"Das Schwert eines Adepten, der damit den Verlu...",The sword first used by an adeptus to make up ...,La espada utilizada por primera vez por una de...,Une épée qui a été initialement conçue pour co...,Pedang yang semula digunakan oleh seorang adep...,兵器で失った力を補おうとする仙人が使っていた剣。無数の妖魔を殺し、最終的に過去の同族をも殺した。,"최초에 선인은 잃어버린 힘을 이 검으로 메꿨다고 한다. 수많은 요괴를 참살하고, 결...",A espada usada pela primeira vez por um Adeptu...,"Меч Адепта, который пытался с его помощью восп...",ดาบที่เคยถูกใช้โดยเซียนเพื่อเสริมพลังในส่วนที่...,Thanh kiếm ban đầu được tiên nhân dùng để bổ s...
13414,blunt-d-d_sword.txt,3,NaN,NaN,測試,Test,Test,Prueba,Test,Ujian,テスト,테스트,Teste,Тест,ทดสอบ,Thử nghiệm


In [46]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'blunt-d_sword.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
993,blunt-d_sword.txt,1,db06c78d1e24cf708a14ce81c9b617ec,测试,磐岩結綠,Moosgrüner Fels,Primordial Jade Cutter,Cortador de Jade Primordial,Coupeur de jade primordial,Primordial Jade Cutter,磐岩の緑,이끼 대검,Cortador de Jade Primordial,Древний Нефритовый Резак,Primordial Jade Cutter,Nham Kết Lục
13415,blunt-d_sword.txt,2,NaN,NaN,苔蘚染綠的岩刀，受了岩之神的勸誡與祝福後變得如翠玉般透徹。,"Ein vom Moos grün gefärbtes Schwert aus Stein,...",A great claymore that was once covered in moss...,Un gran claymore cubierto de musgo que fue ben...,Une épée de roche qui était autrefois recouver...,Pedang besar yang sebelumnya tertutup oleh lum...,コケで緑に染まった岩刀、岩の神の助言と祝福を受けた後、翠玉のように透き通る形になった。,이끼가 파릇파릇한 바위칼은 바위의 신의 훈계와 축복을 받아 옥처럼 투명해졌다,Um grande claymore que já foi coberto de musgo...,"Огромный двуручный меч, который когда-то был п...",ดาบหินที่เต็มไปด้วยตะไคร่น้ำ กลายเป็นสวยใสราวก...,"Thanh đao bị rêu phong bám phủ, sau khi nhận đ..."
13416,blunt-d_sword.txt,3,NaN,NaN,測試,Test,Test,Prueba,Test,Ujian,テスト,테스트,Teste,Тест,ทดสอบ,Thử nghiệm


In [47]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'gewalt-d_pole.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
2325,gewalt-d_pole.txt,1,d41d8cd98f00b204e9800998ecf8427e,,弑主,Göttermörder,Deicide,Cazador de Dioses,Déicide,Deicide,レジサイド,주신 학살창,Deicida,Богоубийство,Deicide,Sát Chủ
13417,gewalt-d_pole.txt,2,NaN,NaN,曾經用來殺死魔神的長槍。其中飽含思念與決意，以及在此之上，能殺死魔神的巨大力量。,Mit dieser Lanze wurden selbst Archonten getöt...,A weapon that was once used to kill an archon....,La lanza que una vez se usó para matar a un di...,Une lance qui a été utilisée pour tuer un dieu...,Sebuah tombak yang konon pernah digunakan untu...,かつて魔神殺しに使われた長槍。中には思いと決意、そしてそれ以上に魔神をも殺せる巨大な力が秘め...,"마신을 무찌를 때 사용되었던 창. 그리움과 결의, 그리고 마신을 죽일 수 있는 거대...",Uma arma que foi usada para derrotar um Arcont...,Этим оружием когда-то убили самого Архонта. Ор...,อาวุธที่ครั้งหนึ่งเคยใช้สังหารเทพมาแล้ว มันเปี...,Thương từng giết chết ma thần. Trong đó ẩn giấ...
13418,gewalt-d_pole.txt,3,NaN,NaN,,,,,,,,,,,,


In [48]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'hunters-d-d-d_bow.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
2394,hunters-d-d-d_bow.txt,1,db06c78d1e24cf708a14ce81c9b617ec,测试,破鏡,Zerbrochener Spiegel,Mirror Breaker,Espejo Roto,Briseur de mirroire,Mirror Breaker,壊れた鏡,부서진 거울,Quebrador de Espelhos,Разбитое зеркало,Mirror Breaker,Gương Vỡ
13419,hunters-d-d-d_bow.txt,2,NaN,NaN,遠古的天外征服者在震撼天地的戰鬥中落下的武器碎片。曾經擁有熄滅月光的力量。,"Das Fragment einer Waffe, zurückgeblieben aus ...",A weapon fragment from the epic battle waged b...,Un fragmento de arma de la épica batalla libra...,Un fragment d'arme laissé par un conquérant ex...,Pecahan senjata yang berasal dari perang yang ...,古代の天外征服者が天地を揺るがす戦いで落とした武器の破片、月の光を消せるほどの力があるらしい。,고대의 정복자가 천지를 뒤흔들던 전투 중에 떨어뜨린 무기의 파편. 달빛을 소멸하는 ...,Um fragmento de arma da batalha épica travada ...,"Часть оружия, использовавшегося в эпических би...",ชิ้นส่วนอาวุธจากมหากาพย์การต่อสู้ยืดเยื้อโดยผู...,Mảnh vũ khí rơi ra trong cuộc chiến đấu kinh t...
13420,hunters-d-d-d_bow.txt,3,NaN,NaN,測試,Test,Test,Prueba,Test,Ujian,テスト,테스트,Teste,Тест,ทดสอบ,Thử nghiệm


In [50]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'hunters-d-d_bow.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
2395,hunters-d-d_bow.txt,1,d41d8cd98f00b204e9800998ecf8427e,,懸黎千鈞,Urzeitlicher Jadebogen,Primordial Jade Vista,Vista de Jade Primordial,Vue de jade primordial,Primordial Jade Vista,高価なオーブ,현려천균,Vista da Jade Primordial,Древняя Нефритовая Виста,Primordial Jade Vista,Cung Huyền Lê
13421,hunters-d-d_bow.txt,2,NaN,NaN,遠古寶玉造的，能守護一國的勁弓，以匯聚一線的翠光為幻造的弦。,Ein aus uralter Jade gefertigter Bogen zum Sch...,"A strong bow that was made from ancient jade, ...",Un gran arco hecho de un ámbar antiguo complet...,Un arc puissant fait de jade antique qui peut ...,Sebuah busur kuat yang tubuhnya terbuat dari g...,上古の宝玉で作られた、一国を守れる弓。一線に集まる光は幻造の弦になる。,고대 보옥으로 만들어진 일국을 수호할 수 있는 강한 활. 푸른빛 줄기가 모여 활시위...,"Um arco forte feito de jade antigo, com uma co...","Лук, сделанный из древнего нефрита, с тетивой,...",คันธนูที่แข็งแกร่งที่ทำจากหยกโบราณด้วยสายที่ทำ...,"Tạo ra từ bảo ngọc viễn cổ, có thể bảo vệ một ..."
13422,hunters-d-d_bow.txt,3,NaN,NaN,,,,,,,,,,,,


In [51]:
# 情况同上
df_testmerge[df_testmerge['文件名'] == 'hunters-d_bow.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
2396,hunters-d_bow.txt,1,d41d8cd98f00b204e9800998ecf8427e,,昆吾墜龍,Kunwus Drachenschlächter,Kunwu's Wyrmbane,Cazador de Dragones de Kunwu,Wyrmbane de Kunwu,Kunwu's Wyrmbane,昆吾の堕龍,곤오추룡,Caça-Dragões de Kunwu,Убийца Драконов Кунву,Kunwu's Wyrmbane,Rồng Kunwu
13423,hunters-d_bow.txt,2,NaN,NaN,名匠昆吾以辰筋做的大弓，其勁可以殲龍。釋弦時，其勢如雷。,Ein großer Bogen aus der Meisterschmiede Kunwu...,A greatbow made from dragon vein. It is so pow...,Un gran arco tan poderoso que podría matar a u...,Un grand arc en veine de dragon fait par le ma...,Busur besar yang terbuat dari nadi naga. Memil...,名匠昆吾が龍の筋を使って作った大弓、龍さえも殺せる強さがある。矢を放つ時、雷のような勢いがある。,용을 죽일 수 있는 힘을 가지고 있는 큰 활. 활시위를 놓을 때의 기세는 벼락과도 같다,Um grande arco feito de veias de dragão. É tão...,"Большой лук, сделанный из драконьей жилы. Из н...",สุดยอดธนูที่ทำมาจากเส้นเลือดของมังกร มันทรงพลั...,Đại cung được thợ thủ công Kunwu làm từ gân cố...
13424,hunters-d_bow.txt,3,NaN,NaN,,,,,,,,,,,,


In [52]:
# 由于这是数据库关联文件，出现的情况与上面相同。鉴于其与主要的文本文件内容具有高度的重复性，故选择删除该文件
df_testmerge[df_testmerge['文件名'] == 'list_weaponsdata_others.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
2582,list_weaponsdata_others.txt,1,df51a444addd3fe0ede06a251010c6e2,无锋剑,無鋒劍,Stumpfes Schwert,Dull Blade,Hoja Desafilada,Épée émoussée,Dull Blade,無鋒の剣,무인검,Lâmina Sem Fio,Тупой меч,Dull Blade,Vô Phong Kiếm
2583,list_weaponsdata_others.txt,2,be03be5bb267d1911e86737a8b0d1caf,银剑,銀劍,Silberschwert,Silver Sword,Espada Plateada,Épée en argent,Silver Sword,銀の剣,실버 소드,Espada de Prata,Серебряный меч,Silver Sword,Kiếm Bạc
2584,list_weaponsdata_others.txt,3,e575651bef0a053393992f3a0b0f35c8,冷刃,冷刃,Kühle Klinge,Cool Steel,Hoja Fría,Lame froide,Cool Steel,冷刃,차가운 칼날,Lâmina Fria,Холодное лезвие,Cool Steel,Lãnh Nhẫn
2585,list_weaponsdata_others.txt,4,09e432636fb02128b06972fbf1e09577,黎明神剑,黎明神劍,Schwert der Dämmerung,Harbinger of Dawn,Espada del Alba,Messager de l'Aube,Harbinger of Dawn,黎明の神剣,여명신검,Prenúncio do Alvorecer,Предвестник зари,Harbinger of Dawn,Thần Kiếm Lê Minh
2586,list_weaponsdata_others.txt,5,444f27bf68277ab2cc903dbe20531c0b,旅行剑,旅行劍,Reiseschwert,Traveler's Handy Sword,Espada del Viajero,Épée du voyageur,Traveler's Handy Sword,旅道の剣,여행자의 검,Espada do Viajante,Меч путешественника,Traveler's Handy Sword,Kiếm Lữ Hành
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13430,list_weaponsdata_others.txt,160,NaN,NaN,破鏡,Zerbrochener Spiegel,Mirror Breaker,Espejo Roto,Briseur de mirroire,Mirror Breaker,壊れた鏡,부서진 거울,Quebrador de Espelhos,Разбитое зеркало,Mirror Breaker,Gương Vỡ
13431,list_weaponsdata_others.txt,161,NaN,NaN,冬極白星,Polarstern,Polar Star,Estrella Invernal,Étoile polaire,Polar Star,冬極の白星,극지의 별,Estrela Invernal,Полярная звезда,Polar Star,Ngôi Sao Cực Đông
13432,list_weaponsdata_others.txt,162,NaN,NaN,若水,Aqua Simulacra,Aqua Simulacra,Aqua Simulacra,Simulacre d'eau,Aqua Simulacra,若水,약수,Aqua Simulacra,Аква симулякрум,Aqua Simulacra,Nhược Thủy
13433,list_weaponsdata_others.txt,163,NaN,NaN,飛雷之弦振,Donnerpuls,Thundering Pulse,Agitador del Relámpago,Pulsation du tonnerre,Thundering Pulse,飛雷の鳴弦,비뢰의 고동,Agitação Trovejante,Громовой пульс,Thundering Pulse,Sấm Sét Rung Động


In [53]:
# 选择删除该文件，理由同上
df_testmerge[df_testmerge['文件名'] == 'charalist_others.txt']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
997,charalist_others.txt,1,822003b8bea68b65457798ee0570b2cf,阿贝多,阿貝多,Albedo,Albedo,Albedo,Albedo,albedo,アルベド,알베도,Albedo,Альбедо,Albedo,Albedo
998,charalist_others.txt,2,c893de60a19ade4f683da44d41bfaca5,埃洛伊,亞蘿伊,Aloy,Aloy,Aloy,Aloy,Albedo,アーロイ,에일로이,Aloy,Элой,Aloy,Aloy
999,charalist_others.txt,3,b4369dd9943f189d2d58ebfd323cc17a,安柏,安柏,Amber,Amber,Amber,Amber,aloy,アンバー,엠버,Amber,Эмбер,Amber,Amber
1000,charalist_others.txt,4,0e9619f81033c66a197ed99e07729974,神里绫华,神里綾華,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Kamisato Ayaka,Aloy,神里綾華,카미사토 아야카,Kamisato Ayaka,Аяка,Kamisato Ayaka,Kamisato Ayaka
1001,charalist_others.txt,5,b8b9bfebdfc7fc2482a05a5a075e17bc,神里绫人,神里綾人,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,Kamisato Ayato,ambor,神里綾人,카미사토 아야토,Kamisato Ayato,Аято,Kamisato Ayato,Kamisato Ayato
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13486,charalist_others.txt,108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yoimiya,NaN,NaN,NaN,NaN,NaN,NaN
13487,charalist_others.txt,109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yunjin,NaN,NaN,NaN,NaN,NaN,NaN
13488,charalist_others.txt,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yun Jin,NaN,NaN,NaN,NaN,NaN,NaN
13489,charalist_others.txt,111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,zhongli,NaN,NaN,NaN,NaN,NaN,NaN


### 4.3 去除重复值
处理缺失值后，根据zh_text_md5字段进行重复数据的检查，进行去重

In [83]:
# 筛选出含有至少一个缺失值的所有行
df_with_missing_values = df_testmerge[df_testmerge.isnull().any(axis=1)]

# 从这些行中提取文件名列，并获取唯一值
unique_files_with_missing_values = df_with_missing_values['文件名'].unique()

# 打印结果
print("包含缺失值的文件名有：")
for file_name in unique_files_with_missing_values:
    print(file_name)


包含缺失值的文件名有：


In [90]:
zh_text_md5_counts = df_testmerge.groupby('zh_text_md5').size().reset_index(
    name='出现次数').sort_values(by='出现次数', ascending=False)

print(zh_text_md5_counts[zh_text_md5_counts.出现次数 >= 2].count())
zh_text_md5_counts[zh_text_md5_counts.出现次数 >= 2]

zh_text_md5    389
出现次数           389
dtype: int64


,zh_text_md5,出现次数
8420,f13ca4d8e5b25516c9e73b8dc5dc1abb,57
7530,d8447cd332e6c2f48f40434bbf493c57,57
4207,794abedba40e72ab68c9d12988300f42,57
7576,d9509db27bfa6e1139d5200a52708797,57
2137,3ceb239ff2d43afa50a2d71cd84993c0,56
...,...,...
1705,3068dabb27f938b0f31340d5a31786bd,2
4789,88a885f9180968d17aca0651b95c1c83,2
7127,cc8e702cc38529c8645af0edc7875d8b,2
4262,7ad51f95d3c5d2175a8f5fa236cdaefb,2


In [89]:
zh_text_md5_counts[zh_text_md5_counts.出现次数 >= 2].count().sum()

778

In [79]:
# 简单查看重复情况
df_testmerge[df_testmerge['zh_text_md5'] == 'f13ca4d8e5b25516c9e73b8dc5dc1abb']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
4983,voice-lines_albedo.txt,89,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5091,voice-lines_aloy.txt,67,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5241,voice-lines_ambor.txt,103,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5397,voice-lines_ayaka.txt,101,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5539,voice-lines_ayato.txt,101,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5671,voice-lines_barbara.txt,93,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5813,voice-lines_beidou.txt,95,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
5953,voice-lines_bennett.txt,93,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
6095,voice-lines_candace.txt,89,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II
6235,voice-lines_chongyun.txt,91,f13ca4d8e5b25516c9e73b8dc5dc1abb,元素战技·其二,元素戰技·其二,Elementarfähigkeit II,Elemental Skill: II,Habilidad Elemental (II),Compétence élémentaire (2),Elemental Skill: II,元素スキル·2,원소전투 스킬·두 번째,Habilidade Elemental - II,Элементальный навык II,สกิลธาตุ - 2,Kỹ Năng Nguyên Tố II


In [80]:
df_testmerge[df_testmerge['zh_text_md5'] == '273472d82748184f4eed1adc63f982ea']

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
1769,dragonfell_claymore.txt,11,273472d82748184f4eed1adc63f982ea,普通攻击与重击命中敌人时，有<color=#99FFFFFF>90%</color>概率在敌...,普通攻擊與重擊命中敵人時，有<color=#99FFFFFF>90%</color>機率在敵...,Erstellt beim Treffen von Gegnern mit Standard...,Hitting an opponent with Normal and Charged At...,Al golpear a los enemigos con Ataques Normales...,Les attaques normales et chargées ont <color=#...,Saat Normal atau Charged Attack mengenai musuh...,通常攻撃と重撃が敵に命中した時、<color=#99FFFFFF>90%</color>の確...,일반 공격과 강공격이 적에게 명중 시 <color=#99FFFFFF>90%</col...,Atingir um inimigo com um Ataque Normal e Ataq...,Попадания обычной или заряженной атакой имеют ...,เมื่อโจมตีปกติหรือชาร์จโจมตีถูกศัตรู จะมีโอกาส...,Khi tấn công thường và trọng kích đánh trúng k...
1847,everfrost_catalyst.txt,11,273472d82748184f4eed1adc63f982ea,普通攻击与重击命中敌人时，有<color=#99FFFFFF>90%</color>概率在敌...,普通攻擊與重擊命中敵人時，有<color=#99FFFFFF>90%</color>機率在敵...,Erstellt beim Treffen von Gegnern mit Standard...,Hitting an opponent with Normal and Charged At...,Al golpear a los enemigos con Ataques Normales...,Les attaques normales et chargées ont <color=#...,Saat Normal atau Charged Attack mengenai musuh...,通常攻撃と重撃が敵に命中した時、<color=#99FFFFFF>90%</color>の確...,일반 공격과 강공격이 적에게 명중 시 <color=#99FFFFFF>90%</col...,Atingir um inimigo com um Ataque Normal e Ataq...,Попадания обычной или заряженной атакой имеют ...,เมื่อโจมตีปกติหรือชาร์จโจมตีถูกศัตรู จะมีโอกาส...,Khi tấn công thường và trọng kích đánh trúng k...
1860,everfrost_pole.txt,11,273472d82748184f4eed1adc63f982ea,普通攻击与重击命中敌人时，有<color=#99FFFFFF>90%</color>概率在敌...,普通攻擊與重擊命中敵人時，有<color=#99FFFFFF>90%</color>機率在敵...,Erstellt beim Treffen von Gegnern mit Standard...,Hitting an opponent with Normal and Charged At...,Al golpear a los enemigos con Ataques Normales...,Les attaques normales et chargées ont <color=#...,Saat Normal atau Charged Attack mengenai musuh...,通常攻撃と重撃が敵に命中した時、<color=#99FFFFFF>90%</color>の確...,일반 공격과 강공격이 적에게 명중 시 <color=#99FFFFFF>90%</col...,Atingir um inimigo com um Ataque Normal e Ataq...,Попадания обычной или заряженной атакой имеют ...,เมื่อโจมตีปกติหรือชาร์จโจมตีถูกศัตรู จะมีโอกาส...,Khi tấn công thường và trọng kích đánh trúng k...


In [87]:
df_testmerge.info

<bound method DataFrame.info of                       文件名  行号                       zh_text_md5  \
0      10001_artedata.txt   1  dd608512c989e96decc20d4a4c53156b   
1      10001_artedata.txt   2  0ec39cfc1b06e6b5ac62dc7e689b1d31   
2      10001_artedata.txt   3  86a111c4a6f097e7354b0616f085d287   
3      10001_artedata.txt   4  9da66b6278278619be64049bbd97d9a6   
4      10001_artedata.txt   5  a55867ecfda45425ca6ad34c1cf33b55   
...                   ...  ..                               ...   
13060  zephyrus_sword.txt   9  e5d6d4d879ba5db47ae21f01962e4a96   
13061  zephyrus_sword.txt  10  44a59492cb7ee50266441565a92a82b1   
13062  zephyrus_sword.txt  11  c842a5cf606543f2133250c0b9dd95da   
13063  zephyrus_sword.txt  12  44a59492cb7ee50266441565a92a82b1   
13064  zephyrus_sword.txt  13  483785363c2e15065573d7733eb00a7d   

                                                  文本_zhs  \
0                                                   行者之心   
1                                          

In [85]:
df_clean = df_testmerge.copy()
# 根据 "zh_text_md5" 列执行去重操作，保留每个重复项的第一个出现
df_clean.drop_duplicates(subset=["zh_text_md5"], keep="first", inplace=True)
df_clean.reset_index(drop=True, inplace=True)
df_clean

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi
0,10001_artedata.txt,1,dd608512c989e96decc20d4a4c53156b,行者之心,行者之心,Fernweh,Resolution of Sojourner,Corazón del Viajero,Cœur du Voyageur,Resolution of Sojourner,旅人の心,행자의 마음,Resolução de Sojourner,Решимость временщика,Resolution of Sojourner,Trái Tim Hành Giả
1,10001_artedata.txt,2,0ec39cfc1b06e6b5ac62dc7e689b1d31,攻击力提高18%。,攻擊力提高18%。,Erhöht den ANG um 18 %.,ATK +18%.,ATQ +18%.,Augmente l'ATQ de 18 %.,ATK +18%.,攻撃力+18%。,공격력+18%,ATQ +18%.,Увеличивает силу атаки на 18%.,เพิ่มพลังโจมตี 18%,Tấn công tăng 18%
2,10001_artedata.txt,3,86a111c4a6f097e7354b0616f085d287,重击的暴击率提升30%。,重擊的暴擊率提升30%。,Erhöht die KT heftiger Schläge um 30 %.,Increases Charged Attack CRIT Rate by 30%.,La Prob. CRIT de los Ataques Cargados aumenta ...,Augmente le taux CRIT des attaques chargées de...,Meningkatkan 30% CRIT Rate dari Charged Attack.,重撃の会心率+30%。,강공격의 치명타 확률+30%,Aumenta a Taxa Crítica do Ataque Carregado em ...,Увеличивает шанс крит. попадания заряженной ат...,เพิ่มอัตราคริของ การชาร์จโจมตีขึ้น 30%,Tỷ lệ bạo kích của trọng kích tăng 30%
3,10001_artedata.txt,4,9da66b6278278619be64049bbd97d9a6,故人之心,故人之心,Sinn der Kameradschaft,Heart of Comradeship,Corazón de la Amistad,Fleur du Voyageur,Heart of Comradeship,故人の心,옛 벗의 마음,Coração de Camaradagem,Сердце товарищества,Heart of Comradeship,Trái Tim Của Cố Nhân
4,10001_artedata.txt,5,a55867ecfda45425ca6ad34c1cf33b55,玉蓝色的小花，花茎上扎着谁人的丝带。,玉藍色的小花，花莖上繫著誰人的絲帶。,Ein hellblaues Blümchen mit einem kleinen Band...,A small cerulean flower with somebody's ribbon...,Una flor azul celeste en cuyo tallo anudaron u...,Une petite fleur bleue avec un ruban attaché à...,"Bunga kecil berwarna biru mutiara, dengan sehe...",藍色の小花、花茎に誰かのリボンがついている。,푸른빛의 작은 꽃. 꽃줄기에 오래된 누군가의 리본이 묶여있다,"Uma pequena flor azul jade, cuja fita está ama...",Маленький цветок с сапфировыми лепестками и пр...,ดอกไม้ขนาดเล็กที่มีสีของหยกสีฟ้า ที่ก้านมีริบบ...,"Bông hoa nhỏ màu xanh ngọc, trên thân có buộc ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8933,zephyrus_pole.txt,2,aea9a7fc7c455f359daf360ea8363700,西风骑士团的制式长枪。枪杆直挺，枪尖轻风流溢。,西風騎士團的制式長槍。槍桿直挺，槍尖輕風流溢。,Eine Lanze in der Machart des Ordo Favonius. S...,A polearm made in the style of the Knights of ...,Una lanza hecha al estilo de los Caballeros de...,Une longue lance généralement maniée par les c...,Polearm yang dibuat berdasarkan gaya dari Knig...,西風騎士団でよく使われる長槍。身が真っ直ぐで、槍先からそよ風が溢れている。,페보니우스 기사단의 제식용 창. 곧게 선 창대에 창 끝으로 산들바람이 넘실댄다,Uma lança no estilo dos Cavaleiros de Favonius...,"Копьё, состоящее на вооружении рыцарей Ордо Фа...",หอกมาตรฐานที่ใช้งานโดย เหล่าอัศวินแห่ง Favoniu...,Thương nghi thức của Đội Kỵ Sĩ Tây Phong. Mũi ...
8934,zephyrus_pole.txt,3,3529f6101c1ad0028358211e5bf482eb,西风骑士团的仪仗长枪，是检阅仪式上骄傲的礼器，亦是对魔物致命的利器。 通过探询风中挺立的乔木...,西風騎士團的儀仗長槍，是檢閱儀式上驕傲的禮器，亦是對魔物致命的利器。 藉由探詢風中挺立的喬木...,Die Favonius-Lanze des Ordo Favonius. Sie wird...,A spear of honor amongst the Knights of Favoni...,Una lanza de honor de los Caballeros de Favoni...,Une lance qui symbolise l'honneur parmi les ch...,Tombak upacara Knights of Favonius adalah senj...,西風騎士団の儀仗用長槍は閲兵儀式で使われる礼器であり、魔物と対抗する武器でもある。 風中を立...,"페보니우스 기사단의 의장용 창, 사열식의 자랑스러운 무기이자 마물에게 치명적인 무기...",Uma lança de honra entre os Cavaleiros de Favo...,Почётное копьё среди рыцарей Ордо Фавониус. Ег...,หอกของกองอัศวินแห่ง Favonius นั้น เป็นอาวุธที่...,Trường thương nghi thức của Đội Kỵ Sĩ Tây Phon...
8935,zephyrus_sword.txt,1,6944377a710a4957e7c5b555a57d6d03,西风剑,西風劍,Favonius-Schwert,Favonius Sword,Espada de Favonius,Épée de Favonius,Favonius Sword,西風剣,페보니우스 검,Espada de Favonius,Меч Фавония,Favonius Sword,Tây Phong Kiếm
8936,zephyrus_sword.txt,2,5a8505d5e4712f9dca4b3560f925ab08,西风骑士团的制式长剑。轻快锐利之余，也能轻易导出元素力量。,西風騎士團的制式長劍。輕快銳利之餘，也能輕易導出元素力量。,Ein Standardlangschwert des Ordo Favonius. Dur...,A standard-issue longsword of the Knights of F...,Una espada larga estándar de los Caballeros de...,Une longue épée généralement maniée par les ch...,Pedang panjang yang digunakan oleh para

## jsonl转换
进行初步数据处理后，将得到的数据框转为jsonl

最终导出的jsonl文件有487行，每一行对应一个文件的内容，共32.7MB

之后执行数据检查，使用数据检查工具

检查工具输出的txt文件参考check_log_multilingual_genshin.txt，格式为平行语料格式，符合目前规范

In [92]:
# 无空缺值，可执行转换
df_clean[df_clean.isnull().any(axis=1)]

,文件名,行号,zh_text_md5,文本_zhs,文本_zht,文本_de,文本_en,文本_es,文本_fr,文本_id,文本_jp,文本_ko,文本_pt,文本_ru,文本_th,文本_vi


In [96]:

grouped = df_clean.groupby('文件名')

output_list = []

time = datetime.datetime.now().strftime('%Y%m%d')

for file_name, group in grouped:
    paragraphs = []

    for i, row in group.iterrows():
        paragraph = {
            '行号': row['行号'],
            '是否重复': False,  # 需根据实际逻辑进行设置检测方法
            '是否跨文件重复': False,  # 同上
            'zh_text_md5': row['zh_text_md5'],
            'zh_text': row['文本_zhs'],
            'en_text': row.get('文本_en', ''),
            'ar_text':'',
            'nl_text':'',
            'de_text': row.get('文本_de', ''),
            'eo_text':'',
            'fr_text': row.get('文本_fr', ''),
            'he_text':'',
            'it_text':'',
            'ja_text': row.get('文本_jp', ''),
            'pt_text': row.get('文本_pt', ''),
            'ru_text': row.get('文本_ru', ''),
            'es_text': row.get('文本_es', ''),
            'sv_text':'',
            'ko_text': row.get('文本_ko', ''),
            'th_text': row.get('文本_th', ''),
            'id_text': row.get('文本_id', ''),
            'vi_text': row.get('文本_vi', ''),
            'cht_text': row.get('文本_zht', ''),           
            'other1_text': '',
            'other2_text': '',
            '扩展字段': '',
            '时间': time
        }
        paragraphs.append(paragraph)
    
    entry = {
        '文件名': file_name,
        '是否待查文件': False,
        '是否重复文件': False,
        '段落数': len(paragraphs),  
        '去重段落数': 0,  #待设置
        '低质量段落数': 0,  # 同上
        '段落': paragraphs,
        '扩展字段': '',
        '时间': time
    }
    output_list.append(entry)

jsonl_str = "\n".join(json.dumps(entry, ensure_ascii=False) for entry in output_list)
with open('multilingual_genshin.jsonl', 'w', encoding='utf-8') as file:
    file.write(jsonl_str)
